In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score



In [ ]:
SEED = 42
os.environ["PYTHONHSSHSEED"] = str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
print("TensorFlow:", tf.__version__)

TensorFlow: 2.19.0


In [ ]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].copy()
y = dataset.iloc[:, -1].values
print("Shapes:", X.shape, y.shape)

Shapes: (10000, 10) (10000,)


In [ ]:
le_gender = LabelEncoder()
X.loc[:, X.columns[2]] = le_gender.fit_transform(X.iloc[:, 2])
ct = ColumnTransformer(
    transformers=[('geo_ohe', OneHotEncoder(handle_unknown="ignore"), [1])],
    remainder="passthrough"
)
X_ohe = ct.fit_transform(X)
X_ohe = np.asarray(X_ohe).astype('float32')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_ohe, y, test_size=0.2, random_state=SEED)

sc = StandardScaler(with_mean=False)
X_train = sc.fit_transform(X_train).astype('float32')
X_test = sc.transform(X_test).astype('float32')

In [ ]:
from tensorflow.keras import layers, models, callbacks
def build_model(input_dim):
  model = models.Sequential([
      layers.Dense(16, activation='relu', input_dim=input_dim),
      layers.Dense(16, activation='relu'),
      layers.Dense(1, activation='sigmoid')
  ])
  model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy', tf.keras.metrics.AUC(name="auc")])
  return model
ann = build_model(input_dim=X_train.shape[1])
ann.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 16)             │           208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 497 (1.94 KB)

 Trainable params: 497 (1.94 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
cb = [
    callbacks.EarlyStopping(patience=10, restore_best_weights=True, monitor="val_auc", mode="max"),
    callbacks.ModelCheckpoint("best_ann.keras", monitor="val_auc", mode="max", save_best_only=True),
    callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5, factor=0.5)
]

In [ ]:
hist = ann.fit(
    X_train, y_train,
    validation_split=0.2, #20% de los datos para validacion y entrenamiento
    epochs=200, #cantidad de veces que compara o ejecuta
    batch_size=32, #32 objetos por bloque
    callbacks=cb,
    verbose=1 #para poder ver como entrena
)

Epoch 1/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.7906 - auc: 0.5954 - loss: 0.5051 - val_accuracy: 0.7987 - val_auc: 0.6992 - val_loss: 0.4651 - learning_rate: 0.0010
Epoch 2/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7919 - auc: 0.6949 - loss: 0.4750 - val_accuracy: 0.8131 - val_auc: 0.7546 - val_loss: 0.4378 - learning_rate: 0.0010
Epoch 3/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8009 - auc: 0.7386 - loss: 0.4524 - val_accuracy: 0.8231 - val_auc: 0.7821 - val_loss: 0.4175 - learning_rate: 0.0010
Epoch 4/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8080 - auc: 0.7645 - loss: 0.4344 - val_accuracy: 0.8306 - val_auc: 0.7967 - val_loss: 0.4048 - learning_rate: 0.0010
Epoch 5/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8138 - auc: 0.7823 - loss: 0.4208 - val_accuracy: 0.8344 - val_auc: 0.8063 - val_loss: 0.3958 - learning_rate: 0.0010
Epoch 6/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8192 - a

In [ ]:
y_proba = ann.predict(X_test).ravel()
y_pred = (y_proba >= 0.5).astype(int)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=4))
print("AUC:", roc_auc_score(y_test, y_proba))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
[[1539   68]
 [ 208  185]]
              precision    recall  f1-score   support

           0     0.8809    0.9577    0.9177      1607
           1     0.7312    0.4707    0.5728       393

    accuracy                         0.8620      2000
   macro avg     0.8061    0.7142    0.7452      2000
weighted avg     0.8515    0.8620    0.8499      2000

AUC: 0.8486076342211476


In [ ]:
sample_raw = pd.DataFrame([{
    "CreditScore": 600,
    "Geography": "France",
    "Gender": "Male",
    "Age": 40,
    "Tenure": 3,
    "Balance": 60000,
    "NumOfProducts": 2,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 50000
}])

sample_raw.loc[:, "Gender"] = le_gender.transform(sample_raw["Gender"])
sample_ohe = ct.transform(sample_raw)
sample_scaled = sc.transform(sample_ohe).astype("float32")

proba = ann.predict(sample_scaled).item()
print(f"\nProbabilidad de churn para el cliente ejemplo: {proba:.4f}")
print("Salir del banco" if proba >= 0.5 else "Permanecer en el banco")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step

Probabilidad de churn para el cliente ejemplo: 0.0511
Permanecer en el banco
